In [30]:
# necessary imports required
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import GridSearchCV, KFold
import pickle as pkl

### Data Preprocessing

In [32]:
df = pd.read_csv("male_players (legacy).csv")

C:\Users\Aggyr\AppData\Local\Temp\ipykernel_8928\2569618666.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("male_players (legacy).csv")


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [34]:
df.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,...,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,...,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,34928.608856,2.559318,0.0,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,...,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,...,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,...,50.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,31.000000
50%,220621.000000,19.000000,2.0,66.000000,70.000000,7.250000e+05,4000.000000,25.000000,181.000000,75.000000,...,59.000000,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000,41.000000
75%,236958.000000,21.000000,2.0,70.000000,75.000000,1.800000e+06,10000.000000,28.000000,186.000000,80.000000,...,66.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000,46.000000
max,271817.000000,23.000000,2.0,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,...,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000,68.000000


In [35]:
df.isnull().sum()

player_id           0
player_url          0
fifa_version        0
fifa_update         0
fifa_update_date    0
                   ..
cb                  0
rcb                 0
rb                  0
gk                  0
player_face_url     0
Length: 110, dtype: int64

In [36]:
# gets all numeric column and replace their NaN values with the mean.
def cleaning_numeric(df):
    df_num = df.select_dtypes(include=[np.number])
    mean_values = df_num.mean()
    numeric = df_num.fillna(mean_values)
    return numeric

In [37]:
# this function combines both one Hot encoding, and filling all NaN values with the mode of the column
def oneHotEconding(df):
    # Select the non-numeric columns
    non_numeric = df.select_dtypes(include = ['object'])
    # fill missing values with the mode
    for col in non_numeric.columns:
        mode_value = non_numeric[col].mode()[0]
        non_numeric[col] = non_numeric[col].fillna(mode_value)

    # OneHot encoding
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded_data = encoder.fit_transform(non_numeric)
    cat = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(non_numeric.columns))
    # returns the encoded dataframe
    return cat

In [38]:
# Main function that ties everything together, cleans the data, 
# fill all Null and NaN values in both numeric and categorical columns
def cleaning_data(df):
    # reads from the csv
    # reason for droping all of these columns was to:
    # 1. to reduce the size of the dataframe, to reduce the noise when training and the time
    # 2. After inspection and research all of these Attribute would not be needed to calculate the overall score of a player
    drop_col = ['player_id', 'player_url','fifa_version','fifa_update','fifa_update_date','short_name','long_name','dob','league_id','club_name','club_team_id',
    'club_jersey_number',
    'club_loaned_from',
    'club_joined_date',
    'club_position',
    'club_contract_valid_until_year',
    'nationality_id','nationality_name','nation_jersey_number','player_face_url', 'real_face','ls','st','rs','lw',
    'lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm',    'rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb',
    'rcb','rb','gk','player_positions','league_name','league_level',
    ]
    
    # drops redudant columns
    drop_col_existing = []
    for col in drop_col:
        if col in df.columns:
            drop_col_existing.append(col)
    
    df = df.drop(drop_col_existing, axis=1)
    # dropping the columns which have null values more than 50%
    L = []
    L_less = []
    for i in df.columns:
        if((df[i].isnull().sum()) < (0.5 * (df.shape[0]))):
            L.append(i)
        else:
            L_less.append(i)
            
    # spliting the numeric columns and filling all NaN values
    numeric = cleaning_numeric(df[L])
    cat = oneHotEconding(df[L])
    return pd.concat([numeric, cat], axis=1)

## Feature Engineering

In [40]:
# pick the top 7 features with the higher corelation
def picking_best_features(df):
    new_df = cleaning_data(df)
    y = new_df['overall']
    X = new_df.drop('overall', axis=1)
    
    scaler= StandardScaler()
    X = scaler.fit_transform(X)

    Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42)

    # used XGBRegressor over random forest because it is faster
    xgb_reg = xgb.XGBRegressor(n_estimators=100, random_state=42)
    xgb_reg.fit(Xtrain, Ytrain)

    feature_importances = pd.Series(xgb_reg.feature_importances_, index=new_df.drop('overall', axis=1).columns)
    predicting_var = feature_importances.sort_values(ascending=False)

    print(predicting_var)

    top10 = predicting_var.head(7)
    return top10

best_features = picking_best_features(df).index.tolist() # picks the names of variables
best_features

value_eur                     0.597714
movement_reactions            0.077989
age                           0.058595
wage_eur                      0.043628
potential                     0.037689
                                ...   
body_type_Normal (170-)       0.000000
body_type_Normal (185+)       0.000000
body_type_Stocky (170-)       0.000000
body_type_Stocky (170-185)    0.000000
body_type_Stocky (185+)       0.000000
Length: 68, dtype: float32


['value_eur',
 'movement_reactions',
 'age',
 'wage_eur',
 'potential',
 'international_reputation',
 'release_clause_eur']

## Training Model
According the rubric Training must be done with either RandomForest, XGBoost and Gradient Boost Regressor. Decided to use all 3.

In [42]:
# spliting the data (uisng the variables with the highest correlation) to test in all the models
df_clean = cleaning_data(df)
X = df_clean[best_features]
y = df_clean['overall']

# split the the best features and the overall to be trained by the models
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42)

In [43]:
# main function to train all the models
def randomForestReg():
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(Xtrain, Ytrain)
    y_pred = rf.predict(Xtest)
    mae = mean_absolute_error(y_pred, Ytest)
    mse = mean_squared_error(y_pred, Ytest)
    rmse = np.sqrt(mean_squared_error(y_pred, Ytest))
    r2 = r2_score(y_pred, Ytest)
    print(f"""
    Mean Absolute Error={mean_absolute_error(y_pred, Ytest)}
    Mean Squared Error={mean_squared_error(y_pred,Ytest)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred, Ytest))}
    R2 score={r2}
    model_name={rf.__class__.__name__}
    """) 
    return {'model': rf, 'MAE': mae}

In [44]:
def xgbReg():
    xg = xgb.XGBRegressor()
    xg.fit(Xtrain, Ytrain)
    y_pred = xg.predict(Xtest)
    mae = mean_absolute_error(y_pred, Ytest)
    mse = mean_squared_error(y_pred, Ytest)
    rmse = np.sqrt(mean_squared_error(y_pred, Ytest))
    r2 = r2_score(y_pred, Ytest)
    print(f"""
    Mean Absolute Error={mean_absolute_error(y_pred, Ytest)}
    Mean Squared Error={mean_squared_error(y_pred,Ytest)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred, Ytest))}
    R2 score={r2}
    model_name={xg.__class__.__name__}
    """) 
    return {'model': xg, 'MAE': mae}

In [45]:
def grad_boost():
    gradReg = GradientBoostingRegressor()
    gradReg.fit(Xtrain, Ytrain)
    y_pred = gradReg.predict(Xtest)
    mae = mean_absolute_error(y_pred, Ytest)
    mse = mean_squared_error(y_pred, Ytest)
    rmse = np.sqrt(mean_squared_error(y_pred, Ytest))
    r2 = r2_score(y_pred, Ytest)
    print(f"""
    Mean Absolute Error={mean_absolute_error(y_pred, Ytest)}
    Mean Squared Error={mean_squared_error(y_pred,Ytest)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred, Ytest))}
    R2 score={r2}
    model_name={gradReg.__class__.__name__}
    """) 
    return {'model': gradReg, 'MAE': mae}

## Evaluation

In [47]:
# Picks the model with the the least mean absolute error, because:
#A high Mean Absolute Error (MAE) score indicates that, on average, 
# the predictions made by the model are significantly different from the actual values.
# Meaning a model with the lowest MAE is likely to be the most accurate.
def picking_best_model():
    rf_result = randomForestReg()
    gd_result = grad_boost()
    xgb_result = xgbReg()
    r2_results = [rf_result, gd_result,xgb_result]
    best_model = min(r2_results, key=lambda x: x['MAE'])
    print(best_model)
    return best_model

In [48]:
best = picking_best_model()


    Mean Absolute Error=0.5151769055604963
    Mean Squared Error=0.7978488257395295
    Root Mean Squared Error=0.8932238385419018
    R2 score=0.983690159198158
    model_name=RandomForestRegressor
    

    Mean Absolute Error=0.9158238697258974
    Mean Squared Error=1.4834586285554079
    Root Mean Squared Error=1.2179731641359788
    R2 score=0.9690193910101702
    model_name=GradientBoostingRegressor
    

    Mean Absolute Error=0.6761840522615731
    Mean Squared Error=0.962041993788713
    Root Mean Squared Error=0.9808373941631269
    R2 score=0.9802623391151428
    model_name=XGBRegressor
    
{'model': RandomForestRegressor(random_state=42), 'MAE': 0.5151769055604963}


In [49]:
# hyper tuning using Grid Search with Cross Validation
def hyper_tuning(model):
    cv = KFold(n_splits=3)

    PARAMETERS_gb = {
        'n_estimators': [100],
        'max_depth': [10, 20],
        'min_samples_split': [2]
    }
    
    model_gs = GridSearchCV(model, param_grid=PARAMETERS_gb, cv=cv, scoring="neg_mean_absolute_error")
    model_gs.fit(Xtrain, Ytrain)
    y_pred = model_gs.predict(Xtest)
    
    # print(model_gs.__class__.__name__, confusion_matrix(Ytest, y_pred), classification_report(Ytest, y_pred))
    return model_gs.best_estimator_
    

In [50]:
tuned_model = hyper_tuning(best['model'])

In [51]:
tuned_model

RandomForestRegressor(max_depth=20, random_state=42)

In [52]:
tuned_model.__class__.__name__

'RandomForestRegressor'

In [53]:
# accessing the tuned models performance
def tuned_model_performance(model):
    model.fit(Xtrain, Ytrain)
    y_pred = model.predict(Xtest)
    mae = mean_absolute_error(y_pred, Ytest)
    mse = mean_squared_error(y_pred, Ytest)
    rmse = np.sqrt(mean_squared_error(y_pred, Ytest))
    r2 = r2_score(y_pred, Ytest)
    print(f"""
    Mean Absolute Error={mean_absolute_error(y_pred, Ytest)}
    Mean Squared Error={mean_squared_error(y_pred,Ytest)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred, Ytest))}
    R2 score={r2}
    model_name={model.__class__.__name__}
    """)
result = tuned_model_performance(tuned_model)
result


    Mean Absolute Error=0.5159443637483515
    Mean Squared Error=0.7876909199935115
    Root Mean Squared Error=0.8875195321757778
    R2 score=0.9838835971259909
    model_name=RandomForestRegressor
    


## Test with new data set

In [69]:
test_df = pd.read_csv("players_22-1.csv")
cln = cleaning_data(test_df)

def testing_tuned_model(model, test_df):
    cln = cleaning_data(test_df)
    best_feature = picking_best_features(cln)
    X = df_clean[best_features]
    y = df_clean['overall']
    Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42)
    y_pred = model.predict(Xtest)

    r2 = r2_score(y_pred, Ytest)
    print(f"""
    Mean Absolute Error={mean_absolute_error(y_pred, Ytest)}
    Mean Squared Error={mean_squared_error(y_pred,Ytest)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred, Ytest))}
    R2 score={r2} """)
    
testing_tuned_model(tuned_model, test_df)
tuned_model.__class__.__name__

C:\Users\Aggyr\AppData\Local\Temp\ipykernel_8928\2242908919.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("players_22-1.csv")


movement_reactions        0.397336
value_eur                 0.385953
age                       0.050109
potential                 0.023656
wage_eur                  0.018459
                            ...   
club_joined_2019-05-24    0.000000
club_joined_2019-05-26    0.000000
club_joined_2019-05-27    0.000000
club_joined_2019-05-28    0.000000
club_joined_2020-03-23    0.000000
Length: 2824, dtype: float32

    Mean Absolute Error=0.5159443637483515
    Mean Squared Error=0.7876909199935115
    Root Mean Squared Error=0.8875195321757778
    R2 score=0.9838835971259909 


'RandomForestRegressor'

In [21]:
# Save the trained model to a pickle file
pkl.dump(tuned_model, open('C:/Users/Aggyr/OneDrive/Desktop/AI_HW/' + tuned_model.__class__.__name__ + '.pkl', 'wb'))
# with open('DenisDemitrus_SportsPrediction.pkl', 'wb') as file:
#     pkl.dump(tuned_model, file)